In [1]:
import Collobrative_Filtering as CF
import numpy as np
import pandas as pd
import redis
import json
import random
import pickle
import inspect
import os

In [2]:
redis_conn = redis.Redis(host='localhost',port=6379,db=0)

## 測試目的：使用演算法的話是否成效會比Random更好

In [3]:
model = CF.COLLOBORATIVE_FILTERING()

In [4]:
with open('tag_mapping.pickle', 'rb') as f3:
    tag_mapping = pickle.load(f3)

In [5]:
def preference_fake(data):
    number = len(data)
    random_lst = []
    for i in range(number):
        if random.randint(0,1)==1:
            random_lst.append(i)
    return random_lst

In [6]:
def preference(number):
    ran_num = random.randint(1,number)
    data = json.loads(redis_conn.get(ran_num))
    model_tags = tag_mapping.keys()
    user_tags = []
    for i in data['batchTag']:
        if i[u'TAG_ID'] in model_tags:
            user_tags.append(tag_mapping[i[u'TAG_ID']])
    return user_tags

## Model Test

In [ ]:
for j in range(1,1501):
    data = json.loads(redis_conn.get(j))
    model_tags = tag_mapping.keys()
    user_tags = []
    for i in data['batchTag']:
        if i[u'TAG_ID'] in model_tags:
            user_tags.append(tag_mapping[i[u'TAG_ID']])
    test = model.predict(user_tags,False,similar_num=5,offertag_num=15,offer_number=1)
    print test

In [ ]:
data = json.loads(redis_conn.get(2))
model_tags = tag_mapping.keys()
user_tags = []
for i in data['batchTag']:
    if i[u'TAG_ID'] in model_tags:
        user_tags.append(tag_mapping[i[u'TAG_ID']])
test = model.predict(user_tags,False,similar_num=5,offertag_num=10,offer_number=1)
print test

In [ ]:
customer_tag = preference_fake(tag_mapping)
customer_reco = model.predict(customer_tag,False,similar_num=10,offertag_num=15,offer_number=3)
customer_reco

## 情境一：視每個顧客為新顧客（描述顧客標籤隨機給予）

### CASE 1：顧客初始不喜歡推薦，Swich推薦的比例

In [7]:
with open('offerlabel_mapping.pickle', 'rb') as f2:
    offerlabel_mapping = pickle.load(f2)
offer_tags = pd.read_csv('OFFERTAG_MAPPING.csv')

In [8]:
def input_data(number, tag_mapping):
    customer = np.zeros(number).tolist()
    for i in range(number):
        cust_tag = preference(1500)
        #tag = np.zeros(len(tag_mapping))
        #tag[[cust_tag]] = 1
        customer[i] = cust_tag
    return customer

In [34]:
def simulation(cust_tag, times, offer_tags, offerlabel_mapping, tag_mapping):
    response = np.zeros([len(cust_tag), times, 3]).tolist()
    for i in range(times):
        for j in range(len(cust_tag)):
            customer_reco = model.predict(cust_tag[j], False, similar_num=10, offertag_num=15, offer_number=3)
            label = offer_tags[offer_tags.OFFER_ID.isin(customer_reco)].LABEL_ID.unique()

            # 令3個被推薦的點擊機率均為0.1，且互相獨立
            if random.random()>=(0.9)**3:
                reco = customer_reco[random.randint(0,2)]
                label = offer_tags[offer_tags.OFFER_ID.isin([reco])].LABEL_ID.unique()
                offerlabel_map = {}
                offerlabel = np.zeros(len(offerlabel_mapping))
                for r in offerlabel_mapping.keys():
                    offerlabel_map[offerlabel_mapping[r]]=r
                for k in label:
                    offerlabel[offerlabel_map[k]] = 1
                rating_table = model.fit(cust_tag[j],offerlabel,True)
                response[j][i][0] = ','.join(customer_reco)
                response[j][i][1] = reco
                response[j][i][2] = 1
                print 'Finish ' + str(j+1) + ' customer ' + str(i+1) + ' times!!' 

            else:
                '''
                label = offer_tags[~offer_tags.OFFER_ID.isin(customer_reco)].LABEL_ID.unique()
                offerlabel_map = {}
                offerlabel = np.zeros(len(offerlabel_mapping))
                for j in offerlabel_mapping.keys():
                    offerlabel_map[offerlabel_mapping[j]]=j
                for k in label:
                    offerlabel[offerlabel_map[k]] = 1
                rating_table = model.fit(tag,offerlabel,True)
                '''
                response[j][i][0] = ','.join(customer_reco)
                print 'Finish ' + str(j+1) + ' customer ' + str(i+1) + ' times!!'  
    return response

In [32]:
sim_data = input_data(100, tag_mapping)

In [33]:
result = simulation(sim_data,10,offer_tags,offerlabel_mapping,tag_mapping)

Finish 1 tag 1 times!!
Finish 2 tag 1 times!!
Finish 3 tag 1 times!!
Finish 4 tag 1 times!!
Finish 5 tag 1 times!!
Finish 6 tag 1 times!!
Finish 7 tag 1 times!!
Finish 8 tag 1 times!!
Finish 9 tag 1 times!!
Finish 10 tag 1 times!!
Finish 11 tag 1 times!!
Finish 12 tag 1 times!!
Finish 13 tag 1 times!!
Finish 14 tag 1 times!!
Finish 15 tag 1 times!!
Finish 16 tag 1 times!!
Finish 17 tag 1 times!!
Finish 18 tag 1 times!!
Finish 19 tag 1 times!!
Finish 20 tag 1 times!!
Finish 21 tag 1 times!!
Finish 22 tag 1 times!!
Finish 23 tag 1 times!!
Finish 24 tag 1 times!!
Finish 25 tag 1 times!!
Finish 26 tag 1 times!!
Finish 27 tag 1 times!!
Finish 28 tag 1 times!!
Finish 29 tag 1 times!!
Finish 30 tag 1 times!!
Finish 31 tag 1 times!!
Finish 32 tag 1 times!!
Finish 33 tag 1 times!!
Finish 34 tag 1 times!!
Finish 35 tag 1 times!!
Finish 36 tag 1 times!!
Finish 37 tag 1 times!!
Finish 38 tag 1 times!!
Finish 39 tag 1 times!!
Finish 40 tag 1 times!!
Finish 41 tag 1 times!!
Finish 42 tag 1 times!!
F

Finish 50 tag 4 times!!
Finish 51 tag 4 times!!
Finish 52 tag 4 times!!
Finish 53 tag 4 times!!
Finish 54 tag 4 times!!
Finish 55 tag 4 times!!
Finish 56 tag 4 times!!
Finish 57 tag 4 times!!
Finish 58 tag 4 times!!
Finish 59 tag 4 times!!
Finish 60 tag 4 times!!
Finish 61 tag 4 times!!
Finish 62 tag 4 times!!
Finish 63 tag 4 times!!
Finish 64 tag 4 times!!
Finish 65 tag 4 times!!
Finish 66 tag 4 times!!
Finish 67 tag 4 times!!
Finish 68 tag 4 times!!
Finish 69 tag 4 times!!
Finish 70 tag 4 times!!
Finish 71 tag 4 times!!
Finish 72 tag 4 times!!
Finish 73 tag 4 times!!
Finish 74 tag 4 times!!
Finish 75 tag 4 times!!
Finish 76 tag 4 times!!
Finish 77 tag 4 times!!
Finish 78 tag 4 times!!
Finish 79 tag 4 times!!
Finish 80 tag 4 times!!
Finish 81 tag 4 times!!
Finish 82 tag 4 times!!
Finish 83 tag 4 times!!
Finish 84 tag 4 times!!
Finish 85 tag 4 times!!
Finish 86 tag 4 times!!
Finish 87 tag 4 times!!
Finish 88 tag 4 times!!
Finish 89 tag 4 times!!
Finish 90 tag 4 times!!
Finish 91 tag 4 

Finish 98 tag 7 times!!
Finish 99 tag 7 times!!
Finish 100 tag 7 times!!
Finish 1 tag 8 times!!
Finish 2 tag 8 times!!
Finish 3 tag 8 times!!
Finish 4 tag 8 times!!
Finish 5 tag 8 times!!
Finish 6 tag 8 times!!
Finish 7 tag 8 times!!
Finish 8 tag 8 times!!
Finish 9 tag 8 times!!
Finish 10 tag 8 times!!
Finish 11 tag 8 times!!
Finish 12 tag 8 times!!
Finish 13 tag 8 times!!
Finish 14 tag 8 times!!
Finish 15 tag 8 times!!
Finish 16 tag 8 times!!
Finish 17 tag 8 times!!
Finish 18 tag 8 times!!
Finish 19 tag 8 times!!
Finish 20 tag 8 times!!
Finish 21 tag 8 times!!
Finish 22 tag 8 times!!
Finish 23 tag 8 times!!
Finish 24 tag 8 times!!
Finish 25 tag 8 times!!
Finish 26 tag 8 times!!
Finish 27 tag 8 times!!
Finish 28 tag 8 times!!
Finish 29 tag 8 times!!
Finish 30 tag 8 times!!
Finish 31 tag 8 times!!
Finish 32 tag 8 times!!
Finish 33 tag 8 times!!
Finish 34 tag 8 times!!
Finish 35 tag 8 times!!
Finish 36 tag 8 times!!
Finish 37 tag 8 times!!
Finish 38 tag 8 times!!
Finish 39 tag 8 times!!


In [48]:
result[0]

[[u'OFF0023,OFF0024,OFF0036', u'OFF0024', 1],
 [u'OFF0023,OFF0024,OFF0036', u'OFF0036', 1],
 [u'OFF0023,OFF0024,OFF0036', 0.0, 0.0],
 [u'OFF0023,OFF0024,OFF0036', 0.0, 0.0],
 [u'OFF0023,OFF0024,OFF0036', 0.0, 0.0],
 [u'OFF0023,OFF0024,OFF0036', 0.0, 0.0],
 [u'OFF0023,OFF0024,OFF0020', u'OFF0024', 1],
 [u'OFF0023,OFF0024,OFF0036', 0.0, 0.0],
 [u'OFF0023,OFF0024,OFF0036', u'OFF0023', 1],
 [u'OFF0023,OFF0024,OFF0036', 0.0, 0.0]]